<a href="https://colab.research.google.com/github/ProfAI/nlp00/blob/master/8%20-%20Chatbot/chatbot_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip

--2019-04-18 14:56:38--  http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.20
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9916637 (9.5M) [application/zip]
Saving to: ‘cornell_movie_dialogs_corpus.zip’

cornell_movie_dialo 100%[===================>]   9.46M  3.10MB/s    in 3.1s    

2019-04-18 14:56:46 (3.10 MB/s) - ‘cornell_movie_dialogs_corpus.zip’ saved [9916637/9916637]



In [2]:
!unzip cornell_movie_dialogs_corpus.zip

Archive:  cornell_movie_dialogs_corpus.zip
   creating: cornell movie-dialogs corpus/
  inflating: cornell movie-dialogs corpus/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/cornell movie-dialogs corpus/
  inflating: __MACOSX/cornell movie-dialogs corpus/._.DS_Store  
  inflating: cornell movie-dialogs corpus/chameleons.pdf  
  inflating: __MACOSX/cornell movie-dialogs corpus/._chameleons.pdf  
  inflating: cornell movie-dialogs corpus/movie_characters_metadata.txt  
  inflating: cornell movie-dialogs corpus/movie_conversations.txt  
  inflating: cornell movie-dialogs corpus/movie_lines.txt  
  inflating: cornell movie-dialogs corpus/movie_titles_metadata.txt  
  inflating: cornell movie-dialogs corpus/raw_script_urls.txt  
  inflating: cornell movie-dialogs corpus/README.txt  
  inflating: __MACOSX/cornell movie-dialogs corpus/._README.txt  


In [2]:
with open("cornell movie-dialogs corpus/movie_lines.txt", encoding="ISO-8859-1") as file:
  lines = file.read().split("\n")
  
len(lines)

304714

In [3]:
for i,line in enumerate(lines):
  print(line)
  if(i>=10):
    break

L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!
L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!
L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.
L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?
L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.
L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow
L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.
L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No
L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I'm kidding.  You know how sometimes you just become this "persona"?  And you don't know how to quit?
L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?
L868 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ The "real you".


In [0]:
lines_dict = {}

for i,line in enumerate(lines):
  try:
    if(len(line)>0):
      line_key = int(line.split()[0][1:])
      line_value = line.split(" +++$+++ ")[-1]
      lines_dict[line_key] = line_value
  except Exception as e:
    print(e)
    print(lines[i])

In [5]:
lines_dict[1045]

'They do not!'

In [0]:
lines_list = sorted(lines_dict.items())

In [7]:
lines_list[:10]

[(49, 'Did you change your hair?'),
 (50, 'No.'),
 (51, 'You might wanna think about it'),
 (59, 'I missed you.'),
 (60, 'It says here you exposed yourself to a group of freshmen girls.'),
 (61, 'It was a bratwurst.  I was eating lunch.'),
 (62, 'With the teeth of your zipper?'),
 (63, 'You the new guy?'),
 (64, 'So they tell me...'),
 (65, "C'mon.  I'm supposed to give you the tour.")]

In [0]:
context = []
target = []

for i in range(1,len(lines_list)):
  
  current_line = lines_list[i]
  prev_line = lines_list[i-1]
    
  if(current_line[0]-1==prev_line[0]):
    context.append(prev_line[1])
    target.append(current_line[1])

In [9]:
context[:10]

['Did you change your hair?',
 'No.',
 'I missed you.',
 'It says here you exposed yourself to a group of freshmen girls.',
 'It was a bratwurst.  I was eating lunch.',
 'With the teeth of your zipper?',
 'You the new guy?',
 'So they tell me...',
 "C'mon.  I'm supposed to give you the tour.",
 'So -- which Dakota you from?']

In [10]:
target[:10]

['No.',
 'You might wanna think about it',
 'It says here you exposed yourself to a group of freshmen girls.',
 'It was a bratwurst.  I was eating lunch.',
 'With the teeth of your zipper?',
 'You the new guy?',
 'So they tell me...',
 "C'mon.  I'm supposed to give you the tour.",
 'So -- which Dakota you from?',
 "North, actually.  How'd you   ?"]

In [0]:
#import string
import re

def preprocess(text):
    
  text = text.lower()
  text = re.sub(r'[^\w\s]','',text)
  #text = text.translate(string.punctuation)
      
  return text

In [49]:
%%time
X = [preprocess(c) for c in context]
Y = [preprocess(t) for t in target]

print(X[0])

did you change your hair
CPU times: user 1.51 s, sys: 9.9 ms, total: 1.52 s
Wall time: 1.52 s


In [42]:
dictionary = set({})

for x in X:
  dictionary|=set(x.split())
  
for y in Y:
  dictionary|=set(y.split())
  
dictionary = list(dictionary)
len(dictionary)

65720

In [43]:
dictionary_rev = {k: v for k, v in zip(dictionary, range(len(dictionary)))}
dictionary_rev["did"]

52325

In [50]:
%%time

for i,x in enumerate(X):
  X[i] = [dictionary_rev[word] for word in x.split()]
  
for i,y in enumerate(Y):
  Y[i] = [dictionary_rev[word] for word in y.split()]
  
print(X[0])

[52325, 43331, 40943, 22214, 32544]
CPU times: user 3.49 s, sys: 134 ms, total: 3.63 s
Wall time: 3.63 s


In [51]:
from keras.preprocessing.sequence import pad_sequences

X = pad_sequences(X)
Y = pad_sequences(Y)

X.shape

(261177, 477)

In [0]:
from keras import model
from keras.layers import Embedding, LSTM, Dropout

model = Sequential()

model.add(Embedding(num_words, 100))
model.add(LSTM(32, dropout=0.5, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(32, dropout=0.5, recurrent_dropout=0.2))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

In [52]:
from keras.layers import Embedding
from keras.layers import Input, Dense, LSTM, TimeDistributed
from keras.models import Model

vocab_size = len(dictionary)
maxLen = 20


# load the pre-trained GloVe vectors into the embedding layer
embed_layer = Embedding(input_dim = vocab_size, output_dim = 50, trainable = True)
embed_layer.build((None,))
#embed_layer.set_weights([embedding_atrix])

# encoder and decoder gloabal LSTM variables with 300 units
LSTM_cell = LSTM(300, return_state = True)
LSTM_decoder = LSTM(300, return_state = True, return_sequences = True)
# final dense layer that uses TimeDistributed wrapper to generate 'vocab_size' softmax outputs for each time step in the decoder lstm
dense = TimeDistributed(Dense(vocab_size, activation = 'softmax'))

input_context = Input(shape = (maxLen, ), dtype = 'int32', name = 'input_context')
input_target = Input(shape = (maxLen, ), dtype = 'int32', name = 'input_target')

# pass the inputs into the embedding layer
input_ctx_embed = embed_layer(input_context)
input_tar_embed = embed_layer(input_target)

# pass the embeddings into the corresponding LSTM layers
encoder_lstm, context_h, context_c = LSTM_cell(input_ctx_embed)
# the decoder lstm uses the final states from the encoder lstm as the initial state
decoder_lstm, _, _ = LSTM_decoder(input_tar_embed, initial_state = [context_h, context_c],)

output = dense(vocab_size, activation = 'softmax')(decoder_lstm)

model = Model([input_context, input_target], output)

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.fit([X, Y], outs, epochs = 10, batch_size = 128)

Instructions for updating:
Colocations handled automatically by placer.


ValueError: ignored